# Shock Tube Analysis Tutorial

This notebook demonstrates how to analyze 1D shock tube (Sod problem) simulations and compare with analytical solutions.

## Contents
1. Loading simulation data
2. Visualizing density, pressure, velocity profiles
3. Comparing with analytical Riemann solution
4. Checking conservation properties
5. Error analysis

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add parent directory to path
import sys
sys.path.append('..')

from readers import SimulationReader
from conservation import ConservationAnalyzer
from theoretical import TheoreticalComparison
from plotting import ParticlePlotter

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load Simulation Data

First, we load the shock tube simulation results. Update the path below to point to your simulation output directory.

In [ ]:
# Path to simulation results
results_dir = Path('../../results/shock_tube')  # Update this path

# Load all snapshots
reader = SimulationReader(results_dir)
snapshots = reader.read_all_snapshots()

print(f"Loaded {len(snapshots)} snapshots")
print(f"Time range: {snapshots[0].time:.4f} to {snapshots[-1].time:.4f}")
print(f"Dimension: {snapshots[0].dim}D")
print(f"Number of particles: {snapshots[0].n_particles}")

## 2. Visualize Final State

Let's plot the density, pressure, velocity, and energy at the final time.

In [ ]:
# Get final snapshot
final_snap = snapshots[-1]

# Create plotter
plotter = ParticlePlotter(figsize=(14, 10))

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle(f'Shock Tube at t = {final_snap.time:.4f}', fontsize=16)

# Plot each quantity
quantities = ['dens', 'pres', 'vel', 'ene']
for ax, qty in zip(axes.flat, quantities):
    plotter.plot_1d(final_snap, quantity=qty, ax=ax)

plt.tight_layout()
plt.show()

## 3. Compare with Analytical Solution

The Sod shock tube has an analytical solution (Riemann problem). Let's compare our simulation with it.

In [ ]:
# Compute analytical solution at final time
gamma = 1.4  # Adiabatic index
x0 = 0.0     # Position of initial discontinuity

# Get analytical solution
x_theory = final_snap.pos[:, 0] - x0
theory = TheoreticalComparison.sod_shock_tube(x_theory, final_snap.time, gamma)

# Compute error
_, error = TheoreticalComparison.compare_shock_tube(final_snap, gamma, x0)
print(f"L2 density error: {error:.6e}")

In [ ]:
# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle(f'Shock Tube Comparison (t = {final_snap.time:.4f})', fontsize=16)

quantities = ['dens', 'pres', 'vel', 'ene']
for ax, qty in zip(axes.flat, quantities):
    plotter.plot_1d(final_snap, quantity=qty, theory=theory, ax=ax)

plt.tight_layout()
plt.show()

## 4. Conservation Analysis

SPH should conserve mass, momentum, and energy (in the absence of dissipation). Let's check these.

In [ ]:
# Compute conservation report
analyzer = ConservationAnalyzer()
report = analyzer.analyze_snapshots(snapshots)

# Print summary
report.print_summary()

In [ ]:
# Plot conservation errors over time
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Conservation Errors', fontsize=16)

# Mass error
axes[0, 0].plot(report.time, report.mass_error)
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('Relative error')
axes[0, 0].set_title('Mass Conservation')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(0, color='k', linestyle='--', alpha=0.3)

# Momentum error
axes[0, 1].plot(report.time, report.momentum_error)
axes[0, 1].set_xlabel('Time')
axes[0, 1].set_ylabel('Relative error')
axes[0, 1].set_title('Momentum Conservation')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].axhline(0, color='k', linestyle='--', alpha=0.3)

# Energy error
axes[1, 0].plot(report.time, report.energy_error)
axes[1, 0].set_xlabel('Time')
axes[1, 0].set_ylabel('Relative error')
axes[1, 0].set_title('Energy Conservation')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(0, color='k', linestyle='--', alpha=0.3)

# Energy components
axes[1, 1].plot(report.time, report.kinetic_energy, label='Kinetic')
axes[1, 1].plot(report.time, report.thermal_energy, label='Thermal')
axes[1, 1].plot(report.time, report.total_energy, label='Total', linestyle='--')
axes[1, 1].set_xlabel('Time')
axes[1, 1].set_ylabel('Energy')
axes[1, 1].set_title('Energy Components')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Error Analysis Over Time

Let's see how the error in the density profile evolves over time.

In [ ]:
# Compute L2 error at each timestep
times = []
errors = []

for snap in snapshots:
    _, error = TheoreticalComparison.compare_shock_tube(snap, gamma, x0)
    times.append(snap.time)
    errors.append(error)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(times, errors, 'o-')
plt.xlabel('Time')
plt.ylabel('L2 Density Error')
plt.title('Error vs Analytical Solution')
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nError statistics:")
print(f"  Initial: {errors[0]:.6e}")
print(f"  Final:   {errors[-1]:.6e}")
print(f"  Maximum: {max(errors):.6e}")
print(f"  Mean:    {np.mean(errors):.6e}")

## 6. Animation (Optional)

Create an animation showing the evolution of the shock tube.

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig, ax = plt.subplots(figsize=(12, 6))

def animate(i):
    ax.clear()
    snap = snapshots[i]
    theory = TheoreticalComparison.sod_shock_tube(
        snap.pos[:, 0] - x0, snap.time, gamma
    )
    plotter.plot_1d(snap, quantity='dens', theory=theory, ax=ax)
    ax.set_ylim([0, 1.2])
    return ax,

anim = FuncAnimation(fig, animate, frames=len(snapshots), interval=200, blit=False)

# Display in notebook
HTML(anim.to_jshtml())

## Summary

This tutorial demonstrated:
- Loading and visualizing shock tube simulation data
- Comparing with the analytical Riemann solution
- Checking conservation of mass, momentum, and energy
- Analyzing errors over time
- Creating animations

### Next Steps
- Try different SPH methods (SSPH, DISPH, GSPH)
- Vary the number of particles
- Experiment with different kernels
- Run with different initial conditions (strong shock, heating/cooling)
- Compare convergence rates